# 5.一等函数
1. 一等对象
    + 运行时创建
    + 可赋值给变量或数据结构中的元素
    + 能作为参数传给函数
    + 能作为函数返回结果
## 5.1 函数是对象
1. 行数对象是function类的实例
2. 

In [7]:
def factorial(n):
    """ return n! """
    return 1 if n < 2 else n * factorial(n-1)

print(factorial.__doc__)
print(type(factorial))
help(factorial)

 return n! 
<class 'function'>
Help on function factorial in module __main__:

factorial(n)
    return n!



## 5.2高阶函数
1. 接受函数为参数，或把函数作为返回结果的函数是高阶函数
2. 经典高阶函数：map、filter、reduce、key，目前可用列表推导式和生成器表达式替代
    + map将某函数分别应用到某序列的元素上，得到对应的函数调用结果映射
    + filter 通过函数过滤某些元素，只保留函数结果为真的元素
    + reduce 将某个函数操作连续应用到某序列元素上，累计结果获得相应的一个值。


In [8]:
print(list(map(factorial, range(6))))
print([factorial(n) for n in range(6)])#列表推导式实现map

print(list(map(factorial,filter(lambda n: n%2, range(6)))))
print([factorial(n) for n in range(6) if n%2])#通过if实现filter

[1, 1, 2, 6, 24, 120]
[1, 1, 2, 6, 24, 120]
[1, 6, 120]
[1, 6, 120]


In [1]:
from functools import reduce
from operator import add
print(reduce(add, range(100)))
print(sum(range(100)))

4950
4950


## 5.3匿名函数
1. lambda函数的定义体只能使用纯表达式，不能赋值，使用while、try
2. 匿名函数重构：
    + (1)编写注释，说明 1ambda 表达式的作用。
    + (2）研究一会儿注释，并找出一个名称来概括注释。
    + (3) 把 1ambda 表达式转换成 def 语句，使用那个名称来定义函数。
    + (4) 删除注释。
[Functional Programming HOWTO](https://docs.python.org/3/howto/functional.html)

## 5.4可调用对象
可使用callable()判断对象是否可以调用
1. 用户定义的函数
    使用 def 语句或 lambda 表达式创建
2. 内置函数
    使用C语言(CPython) 实现的函数，如 len 或 time.strftime。
3. 内置方法
    使用 C语言实现的方法，如 dict.get。
4. 方法
    在类的定义体中定义的函数。
5. 类
    调用类时会运行类的_new_方法创建一个实例，然后运行_init_方法，初始化实例，最后把实例返回给调用方。
6. 类的实例
    如果类定义了_\_call__方法，其实例可以作为函数调用
7. 生成器函数
    使用了yield关键字的函数或方法。调用生成器函数返回的是生成器对象

## 5.5用户定义的可调用类型
1. 只要对象的类实现了_\_call__方法，对象都可以被调用

In [5]:
import random
class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    def __call__(self):
        return self.pick()

bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo())#直接调用对象与调用pick方法效果一样
print(callable(bingo))

0
1
True


## 5.6函数内省
1. 通过dir()函数获取对象或类所有属性，包含_\_dict__属性以及从父类继承的属性
2. _\_dict__属性只获取当前实例的属性（不包含类、父类的属性）
    + [What's the biggest difference between dir() and __dict__ in Python](https://stackoverflow.com/questions/14361256/whats-the-biggest-difference-between-dir-and-dict-in-python)

In [16]:
class Food(object):
    bar = "spam"
f = Food()
print(Food.__dict__, f.__dict__)
Food.ham = "eggs"
print(Food.__dict__['ham'], f.__dict__)
print(f.ham)

f.fruit = 'apple'
print('fruit' in Food.__dict__, f.__dict__)

{'__module__': '__main__', 'bar': 'spam', '__dict__': <attribute '__dict__' of 'Food' objects>, '__weakref__': <attribute '__weakref__' of 'Food' objects>, '__doc__': None} {}
eggs {}
eggs
False {'fruit': 'apple'}


## 5.7 从定位参数到仅限关键字参数
1. Python提供灵活的参数处理机制。调用函数时使用* 和 ** 展开可迭代对象，映射到单个参数
2. 仅限关键字参数需要放在前面有*的参数后面，若不想支持数量不定的定位参数，可在函数签名里只放一个\*

In [24]:
def f(a,*,b):
    return a
f(1)#强制传入仅限关键参数b，否则会报错

TypeError: f() missing 1 required keyword-only argument: 'b'

In [50]:
def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags
    1、第一个参数name，可以是作为定位参数传入也可以作为关键字参数传入
    2、第一个参数后面任意个参数会被*content捕获，存入一个元组
    3、第三个参数cls仅限关键字参数只能作为关键字参数传入
    4、当传入函数签名中未命名的关键字参数时，会被**attrs捕获
    """
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

print(tag('p','hello','word',title='hello'))#title 被**attrs捕获
my_tag  = {'name':'img','title':'Sunset Boulevard','src':'sunset.jpg', 'cls':'framed' }
print(tag(**my_tag))

<p title="hello">hello</p>
<p title="hello">word</p>
<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />


## 5.8获取函数参数信息
1. _\_default__ 属性 保存定位参数、关键字参数的默认值，返回一个元组
2. _\_kwdefaults__属性 保存仅限关键字参数的默认值，返回一个字典
3. 参数的名称保存在_\_code__ 属性 co_varnames

In [35]:
print(tag.__defaults__)#获取参数默认值
print(tag.__kwdefaults__)#获取仅限关键字参数默认值
print(tag.__code__)
print(tag.__code__.co_varnames)#返回所有类型参数
print(tag.__code__.co_argcount)#返回参数个数，不包括*args **kwargs
print(tag.__code__.co_kwonlyargcount)#返回仅限关键字参数个数，不包括*args **kwargs

('p',)
{'cls': None}
<code object tag at 0x7fb4117a0b30, file "/var/folders/ty/dsj71tyj60v5x5mrmdh35fwc0000gn/T/ipykernel_3375/1824390426.py", line 1>
('name', 'cls', 'content', 'attrs')
1
1


4. inspect模块——提取函数签名
5. inspect.ParameterKind
    + POSITIONAL_OR KEYWNORD
        可以通过定位参数和关键字参数传入的形参（多数 Python 函数的参数属于此类）。
    + VAR_ POSITIONAL
        定位参数元组。
    + VAR KEYWORD
        关键字参数字典。
    + KEYWORD ONLY
        仅限关键字参数
    + POSITIONAL_ONLY
        仅限定位参数；目前，Pyhon 声明函数的句法不支持，但是有些使用C语言实现且不接受关键字参数的函数（如 divmod）支持。
6. 通过inspect.Signature 的bind方法可以将参数绑定到函数签名的形参上，可以在调用函数前验证参数是否合法


In [41]:
from inspect import signature
sig = signature(tag)
print(repr(sig))#返回一个signature对象
print(sig.parameters)#parameters 为一个orderedDict对象
for name, param in sig.parameters.items():
    print(param.kind, ":", name, '=', param.default)

<Signature (name='p', *content, cls=None, **attrs)>
OrderedDict([('name', <Parameter "name='p'">), ('content', <Parameter "*content">), ('cls', <Parameter "cls=None">), ('attrs', <Parameter "**attrs">)])
POSITIONAL_OR_KEYWORD : name = p
VAR_POSITIONAL : content = <class 'inspect._empty'>
KEYWORD_ONLY : cls = None
VAR_KEYWORD : attrs = <class 'inspect._empty'>


In [51]:
import inspect
sig = inspect.signature(tag)
my_tag  = {'name':'img','title':'Sunset Boulevard','src':'sunset.jpg', 'cls':'framed' }
bound_args = sig.bind(**my_tag)
print(repr(bound_args))
for name,value in bound_args.arguments.items():
    print(name, '=', value)
del my_tag['name']
bound_args = sig.bind(**my_tag)#缺少需要的参数，无法绑定。抛出TypeError


<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>
name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


TypeError: missing a required argument: 'name'